# Práctica 4: Umbralización

En esta práctica vamos a programar algunos de los algoritmos vistos en clase sobre umbralización. El objetivo es seguir afianzando los conocimientos de programación y tener una visión más clara sobre qué hace cada uno de los algoritmos vistos en clase

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def calculaHistograma(imagen):
    histograma = np.zeros((256))
    filas, columnas = imagen.shape
    for i in range(filas):
        for j in range(columnas):
            histograma[imagen[i,j]] += 1
    return np.uint8(histograma)

### Ejercicio 1: mínimo entre máximos

El primer programa que vamos a implementar es la umbralización de mínimo entre máximos. Para ello, crea una función que reciba como parámetro de entrada una imagen. El programa obtendrá el histograma de la imagen. A partir del histograma es muy sencillo obtener la posición que ocupa el máximo global del histograma (recuerda la función argmax). El siguiente paso es encontrar el segundo pico del histograma. Recuerda que no es tan sencillo como buscar el segundo máximo del histograma, sino que tenemos que recorrer cada una de las intensidades:
- calcular el pico del histograma $q_1=\arg \max_i h(i)$
- para cada intensidad i:
    - Calcular $h(i)\times |i-q_1|$
- tomar como $q_2$  la posición que maximice la expresión anterior

Una vez localizados $q_1,q_2$, busca el mínimo del histograma entre ambos picos. Umbraliza la imagen utilizando dicho umbral. Recuerda que puedes hacer algo como
 $$umbralizada = 255*(imagen<=q).astype(np.uint8)$$
 

Calchist devuelve columna

In [5]:
def min_entre_max(imagen):
#     histograma = calculaHistograma(imagen) #cambiar
    histograma = cv2.calcHist([imagen],[0],None,[256],[0,256]).reshape(-1)
    i = np.arange(0,256,1)
    q1 = np.argmax(histograma)
    print(q1)
    q2 = np.argmax(histograma*np.abs(i-q1))
    print(q2)
    if q1>q2:
        q = np.argmin(histograma[q2:q1+1]) + q2
    else:
        q = np.argmin(histograma[q1:q2+1]) + q1 #estas encontreando la posiciion entre q1 y q2 asi q hay q sumarle la menor para tener la original
    umbralizada = 255 * (imagen<= q).astype(np.uint8)
    return umbralizada

Una vez programada llama a la función con la imagen 01.bmp. Si devuelves el primer mínimo que encuentres, deberías tener como umbral $q=56$ ¿Obtienes un buen resultado? ¿Por qué? ¿Qué pegas encuentras?

In [6]:
# imagen = cv2.imread('images/01.bmp', 0)
imagen = cv2.imread('01.bmp', 0)
umbralizada = min_entre_max(imagen)
cv2.imshow('Umbralizacion minimo entre maximos', umbralizada)
cv2.waitKey(0)
cv2.destroyAllWindows()

53
253


### Ejercicio 2: Otsu

Realizar una función en Matlab que implemente el algoritmo de Otsu. Para ello, vamos a crear un array vacío de 256 posiciones. En la posición i-ésima del vector almacenaremos la varianza entre clases $\sigma_b(i)$ tomando como umbral el valor de intensidad i. Por último obtenendremos como umbral aquel asociado a la máxima varianza entre clases. Recuerda que para calcular la varianza entre clases debemos tomar un umbral i y calcular:
- $p_1$: probabilidad de la clase 1
- $p_2$: probabilidad de la clase 2
- $\mu_1$: media de la clase 1
- $\mu_2$: media de la clase 2
- varianza entre clases: $\sigma_b(i)=p_1p_2(\mu_1- \mu_2)^2$

Estos cálculos se pueden hacer de dos maneras distintas. Una opción sería utilizando funciones de numpy como np.mean, etc. Otra opción sería utilizar el histograma de la imagen y calcular $p_1,p_2,\mu_1,\mu_2$ utilizando únicamente el histograma.

    Necesitamos probabilidad, media, varianza ( p0*p1*(meanClase0 - meanClase2)**2)umbral el argmax de las varianzas

In [7]:
def otsu(imagen):
    r,c = imagen.shape
    #reshape(-1,1) (-1) -> para lo q halla , (1) -> una columna
    #reshape(-1) (-1) -> para lo q halla , ( ) -> 0 columnas
    histograma = cv2.calcHist([imagen],[0],None,[256],[0,256]).reshape(-1)
    i = np.arange(0,256,1)
    o = np.zeros(histograma.size-1)
    for x in range(o.size):
        p0 = np.sum(histograma[0:x+1])/(r*c)
        p1 = 1 - p0
#         m0 = np.mean(imagen[imagen<=x])
#         m1 = np.mean(imagen[imagen>x])
        m0 = np.sum(histograma[0:x+1]*i[0:x+1])/(np.sum(histograma[0:x+1]))
        m1 = np.sum(histograma[x+1:]*i[x+1:])/(np.sum(histograma[x+1:]))
        o[x] = p0*p1*((m0-m1)**2)
    q = np.argmax(o)
    umbralizada = 255 * (imagen<= q).astype(np.uint8)
    return umbralizada

Una vez programada, prueba la umbralización con la imagen 01.bmp.

Compara las imagenes obtenidas con el Ejercicio 1 y el Ejercicio 2. ¿Por qué crees que el resultado del Ejercicio 1 no tiene tanta calidad?

Prueba ahora a suavizar la imagen con un filtro Gaussiano (o un filtro de la media 3x3 o 5x5) y luego umbraliza con la función Otsu. ¿Obtienes alguna mejora?


In [8]:
# imagen = cv2.imread('images/01.bmp', 0)
imagen = cv2.imread('01.bmp', 0)
umbralizada = otsu(imagen)
cv2.imshow('Umbralizacion Otsu', umbralizada)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Umbralizació de mínimo error

Este algoritmo tiene algo parecido al anterior. Vamos a ir probando cada uno de las intensidades posibles como umbral y, para cada una, calcularemos una distribución de probabilidad gaussiana según la media y la desviación de los píxeles de la clase 0 y clase 1 en función del umbral. Para ello, recorre el conjunto de intensidades y calcula:
- $p_0$: probabilidad de la clase0
- $p_1$: probabilidad de la clase1
- $\mu_0$: media de la clase 0
- $\mu_1$: media de la clase 1
- $\sigma_0$: varianza de la clase 0
- $\sigma_1$: varianza de la clase 1

Una vez tenemos los parámetros de cada una de las clases, vamos a calcular el error que cometemos al asignar cada una de las intensidades a la clase 0 y a la clase 1 en función del umbral. Esto se calcula como

$$e(q)=\sum_{i=0}^{q}h_p(i)e_0(i)+\sum_{i=q+1}^{255}h_p(i)e_1(i)$$

donde

$$e_j(i)=\frac{(i-\mu_j)^2}{\sigma_j^2}+2(\log(\sigma_j)-\log(p_0))$$

Una vez calculado el error por cada posible umbral, tomaremos aquel que minimice el error cometido.

In [9]:
def minimo_error(imagen):
    r,c = imagen.shape
    #reshape(-1,1) (-1) -> para lo q halla , (1) -> una columna
    #reshape(-1) (-1) -> para lo q halla , ( ) -> 0 columnas
    histograma = cv2.calcHist([imagen],[0],None,[256],[0,256]).reshape(-1)
    i = np.arange(0,256,1)
    error = np.zeros(histograma.size-1)
    for x in range(0,error.size):
        p0 = np.sum(histograma[0:x+1]/(r*c))
        p1 = 1 - p0
#         m0 = np.mean(imagen[imagen<=x])
#         m1 = np.mean(imagen[imagen>x])
        m0 = np.sum(histograma[0:x+1]*i[0:x+1])/(np.sum(histograma[0:x+1]))
        m1 = np.sum(histograma[x+1:]*i[x+1:])/(np.sum(histograma[x+1:]))
        o0 = np.sum(((i[0:x+1]-m0)**2)*histograma[0:x+1])/(np.sum(histograma[0:x+1]))
        o1 = np.sum(((i[x+1:]-m1)**2)*histograma[x+1:])/(np.sum(histograma[x+1:]))
        if o0==0:
            e0 = 0
        else:
            e0 = ((i[0:x+1]-m0)**2)/(o0**2)+2*(np.log(o0)-np.log(p0))
        if o1==0:
            e1 = 0
        else:
            e1 = (i[x+1:]-m1)**2/(o1**2)+2*(np.log(o1)-np.log(p1))
        error[x] = np.sum(histograma[0:x+1]*e0)/(r*c)+np.sum(histograma[x+1:]*e1)/(r*c)
    q = np.argmin(error)
    umbralizada = 255 * (imagen<= q).astype(np.uint8)
    return umbralizada

Una vez programado, comprueba su funcionamiento con las imágenes de antes. Al igual que aparece en las explicaciones de clase, podríamos pintar encima del histograma las dos distribuciones gaussianas que tratan de ajustarse al histograma. Para ello, una vez conocido el umbral q, las distribcuines gaussianas de la clase 0 y de la clase 1 vendrían dadas, para cada posible intensidad x, por

$$\frac{1}{\sqrt{2\pi\sigma_0^2}}*\exp\left(-\frac{(x-\mu_0)^2}{2\sigma_0^2}\right)$$
$$\frac{1}{\sqrt{2\pi\sigma_1^2}}*\exp\left(-\frac{(x-\mu_1)^2}{2\sigma_1^2}\right)$$

In [10]:
# imagen = cv2.imread('images/01.bmp', 0)
imagen = cv2.imread('01.bmp', 0)
umbralizada = minimo_error(imagen)
cv2.imshow('Umbralizacion Otsu', umbralizada)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Ejercicio 3: Umbralización variable

Uno de los problemas del algoritmo de Otsu aparece cuando la iluminación de la imagen no es constante. Lee la imagen texto.tif. Visualiza y aplica el algoritmo del ejercicio 2 (otsu). Comprueba que la segmentación no es la adecuada. Vamos a solventar este problema mediante una umbralización variable, es decir, una combinación de técnicas globales y locales. De hecho, vamos a ver que se parece mucho a un filtro no lineal sobre la imagen. El algoritmo es el siguiente:
- para cada píxel de la imagen
    - tomar una ventana 3x3 de píxeles centrada en el
    - calcular la media aritmética $\mu_{x,y}$ y la desviación típica $\sigma_{x,y}$
    - Realizar la siguiente segmentación 
    $$g(x,y)=\begin{cases} 
        1 & \mbox{ si } imagen(x,y)<\sigma_{x,y}+0.5\mu_{x,y} \\
        0 & \mbox{ en otro caso}
    \end{cases}$$

In [30]:
# imagen = cv2.imread('images/texto.tif', 0)
imagen = cv2.imread('texto.tif', 0)
texto_umbralizada = otsu(imagen)
print(texto_umbralizada)
cv2.imshow('Imagen texto', texto_umbralizada)
cv2.waitKey(0)
cv2.destroyAllWindows()



[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


In [25]:
def variable(imagen):
    umbralizada = np.ones(np.shape(imagen))
    imagen_ampliada = cv2.copyMakeBorder(imagen, 1, 1, 1, 1, cv2.BORDER_REPLICATE)
    for i in range(1,np.shape(imagen)[0]+1):
        for j in range(1,np.shape(imagen)[0]+1):
            vecindad = imagen_ampliada[i-1:i+1,j-1:j+1]
            mean = np.mean(vecindad)
            o = (np.sum(((vecindad.flatten()-mean))**2)/vecindad.size)**(0.5)
            if imagen_ampliada[i,j]<(o+0.5*mean):
                umbralizada[i-1,j-1] = 1
            else:
                umbralizada[i-1,j-1] = 0
    return np.uint8(umbralizada)

Comprueba ahora que la umbralización variable funciona bien en este ejemplo concreto. Evidentemente, la fórmula elegida es ad-hoc para la imagen concreta, pero en función de la iluminación podríamos elegir otras condiciones del estilo 
$$imagen(x,y)<a \sigma_{x,y} + b \mu_{x,y}$$
siendo $a$ y $b$ dos números cualesquiera.

In [26]:
imagen = cv2.imread('texto.tif', 0)
# imagen = cv2.imread('images/texto.tif', 0)
texto_umbralizada_2 = variable(imagen)*255
cv2.imshow('Umbralizacion Variable', texto_umbralizada_2)
cv2.waitKey(0)
cv2.destroyAllWindows()